In [33]:
#create the required directory

base_dir = "Training data processed"

import os

os.mkdir(base_dir)

categories = ['Pylon', 'Twilight']

for i in categories:
    try:
        #os.system("sudo mkdir Pylon")
        os.mkdir(base_dir + '/%s' % i)
    except OSError as e:
        print(e)

FileExistsError: [Errno 17] File exists: 'Training data processed'

In [34]:
from PIL import Image
import os 
import numpy as np

counter = 0

def reformat_training_images():
    
    categories = ['Pylon', 'Twilight']
    
    original_data_folder = "Training data cropped"
    target_data_folder = "Training data processed"
    
    for category in categories: 
        #current folder, e.g., "Training data cropped/Pylon"
        current_folder = original_data_folder + "/" + category
        counter = 1        
        
        #iterate through all the images in training_data_cropped
        for filename in os.listdir(current_folder):

            if filename.endswith(".jpg") or filename.endswith(".png"):
                img = Image.open(current_folder + "/" + filename)
                #theoretically should reshape but here they're all 250, 250
                img = img.convert("RGB")
                img.save(target_data_folder + "/" + category + "/" + "image" + str(counter) + ".png")
                counter += 1

reformat_training_images()

In [45]:
# Here we will search for the python script im2rec
import sys,os

base_dir = "Training_data_processed"

suffix='incubator-mxnet/tools/im2rec.py'
im2rec = list(filter( (lambda x: os.path.isfile(x + suffix )), sys.path))[0] + suffix
%env IM2REC=$im2rec
%env BASE_DIR=$base_dir

env: IM2REC=incubator-mxnet/tools/im2rec.py
env: BASE_DIR=Training_data_processed


In [47]:
%%bash

# Ok. Here, im2rec will read all the images and create two .lst files, one for training and other for validation
# this files will then be used for creating the RecordIO files

#cd $BASE_DIR
python $IM2REC --list --recursive --test-ratio=0.2 --train-ratio=0.8 protoss_building training_data_processed/
ls *.lst

Pylon 0
Pylon/.ipynb_checkpoints 1
Twilight 2
protoss_building_test.lst
protoss_building_train.lst


In [50]:
%%bash 

python $IM2REC --num-thread=4 --pass-through protoss_building_train.lst Training_data_processed
python $IM2REC --num-thread=4 --pass-through protoss_building_test.lst Training_data_processed
ls *.rec

Creating .rec file from /Users/emmanuelsevrin/Desktop/sc2_advisor/Model_training/protoss_building_train.lst in /Users/emmanuelsevrin/Desktop/sc2_advisor/Model_training
time: 0.011706113815307617  count: 0
Creating .rec file from /Users/emmanuelsevrin/Desktop/sc2_advisor/Model_training/protoss_building_test.lst in /Users/emmanuelsevrin/Desktop/sc2_advisor/Model_training
time: 0.005207061767578125  count: 0
protoss_building_test.rec
protoss_building_train.rec
